In [12]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math
import time
from torch.utils.data import DataLoader, TensorDataset
import itertools

In [13]:
import sys
root = '../../../'
sys.path.append(root)
from HelpfulFunctions.batchCreation import createBatch
from HelpfulFunctions.metrics_final import meanAveragePrecisionOptimized
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
def CreateDataset(root, num_classes, batch_size, train = True):
    if train == True:
        #Create X_train_tensor
        X_train = np.load( root + "Features/X_hpo_Img.npy" ) # Shape = (45000, 4096)
        X_train_tensor = torch.tensor(X_train)

        #Create Y_train_tensor
        y_train = np.load( root + "Features/y_hpo_Img.npy" ) # Shape = (45000,)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        y_train_tensor = torch.nn.functional.one_hot(y_train_tensor)#, num_classes) #One-Hot Encoded -> Shape = (45000, num_classes)

        #Create indices
        indices_train = torch.arange(len(X_train_tensor))

        dataset = TensorDataset(X_train_tensor, y_train_tensor, indices_train)
        train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return train_loader

    else:
        X_test = np.load( root + "Features/X_val_Img.npy" ) # Shape = (10000, 4096)
        X_test_tensor = torch.tensor(X_test)

        y_test = np.load( root + "Features/y_val_Img.npy" ) # Shape = (10000,)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)
        y_test_tensor = torch.nn.functional.one_hot(y_test_tensor)#, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

        #Create indices
        indices_test = torch.arange(len(X_test_tensor))

        dataset = TensorDataset(X_test_tensor, y_test_tensor, indices_test)
        test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return test_loader

    #Missing implementation for Test and Validation
    

In [15]:
train_loader = CreateDataset(root, num_classes = 10, batch_size = 128)
test_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = False)


In [16]:
class CustomNN(nn.Module):
    def __init__(self, bits):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 1024),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(1024, bits),    # Second fully connected layer to reduce to 4000
        )

    def forward(self, x):
        return self.fc_layers(x)

In [17]:
class DSHLoss(torch.nn.Module):
    def __init__(self, train_size, n_classes, bit):
        super(DSHLoss, self).__init__()
        self.m = 2 * bit
        self.U = torch.zeros(train_size, bit).float().to(device)
        self.Y = torch.zeros(train_size, n_classes).float().to(device)

    def forward(self, u, y, ind, eta):
        self.U[ind, :] = u.data
        self.Y[ind, :] = y.float()

        dist = (u.unsqueeze(1) - self.U.unsqueeze(0)).pow(2).sum(dim=2)
        y = (y @ self.Y.t() == 0).float()

        loss = (1 - y) / 2 * dist + y / 2 * (self.m - dist).clamp(min=0)
        loss1 = loss.mean()
        loss2 = eta * (1 - u.abs()).abs().mean()

        return loss1 + loss2

In [18]:
# Define the grid

def DSH(device: torch.device, train_size: int, n_classes: int, bit: int, num_epoch: int, batch_size: int, eta_values: list, wd_values: list, lr_values: list):

    train_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 1)
    #test_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 2)
    validation_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 2)




    param_grid = {
        'eta': eta_values,
        'learning_rate': lr_values,
        #'batch_size': [16, 32, 64],
        'weight_decay': wd_values
    }

    customLoss = DSHLoss(train_size, n_classes, bit)


    # Get all combinations of parameters
    keys, values = zip(*param_grid.items())
    parameter_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # Evaluate each parameter combination
    best_params = None
    best_map = 0

    for params in parameter_combinations:
        print(f"Testing combination: {params}")
    
        # Initialize loss function with specific parameters
        #loss_fn = customLoss()

        # Initialize model and optimizer
        model = CustomNN(bits = bit).to(device)
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'], weight_decay = params['weight_decay'])

        # Train the model
        for epoch in range(num_epoch):  # Example epoch count
            current_time = time.strftime('%H:%M:%S', time.localtime(time.time()))
            print("%s[%2d/%2d][%s] bit:%d, dataset:%s, training...." % (
            "DPSH", epoch + 1, num_epoch, current_time, bit, "CIFAR"), end="")
            model.train()
            train_loss = 0
            for image, label, ind in train_loader:
                image = image.to(device)
                label = label.to(device)

                optimizer.zero_grad()
                u = model(image)

                loss = customLoss(u, label.float(), ind, eta = params['eta'])
                train_loss += loss.item()

                loss.backward()
                optimizer.step()

            train_loss = train_loss / (train_size / batch_size)
            print("\b\b\b\b\b\b\b loss:%.5f" % (train_loss))
        # Validate the model
        model.eval()
        hash_train = []
        label_train = []

        hash_val = []
        label_val = []

        with torch.no_grad():
            for image, label, ind in train_loader:
                image = image.to(device)
                label_batch = label.to(device)
                hash_train_batch = (model(image)).sign()
                hash_train.append(hash_train_batch)
                label_train.append(label_batch)
                #hash_codes_matrix = np.vstack(hash_code_batches)
                #print(hash_codes_matrix.shape) 

            hash_train = torch.cat(hash_train, dim = 0).cpu().numpy()
            label_train = torch.cat(label_train, dim = 0).cpu().numpy()

            
            for image, label, ind in validation_loader:
                image = image.to(device)
                label_batch = label.to(device)
                hash_val_batch = (model(image)).sign()
                hash_val.append(hash_val_batch)
                label_val.append(label_batch)
            hash_val = torch.cat(hash_val, dim = 0).cpu().numpy()
            label_val = torch.cat(label_val, dim = 0).cpu().numpy()

            map = meanAveragePrecisionOptimized(training_hashes = hash_train, training_labels = label_train, test_hashes = hash_val, test_labels = label_val)

            print(f"Validation mAP: {map:.4f}")

            # Update best parameters
            if map > best_map:
                best_map = map
                best_params = params

    print("Best Parameters:", best_params)
    print("Best mAP:", best_map)
    return best_params

In [ ]:
#DSH12Img = DSH(device, 50000, 1001, 12, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])
#DSH24Img = DSH(device, 50000, 1001, 24, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])
DSH32Img = DSH(device, 50000, 1001, 32, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])
#DSH48Img = DSH(device, 50000, 1001, 48, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])

Testing combination: {'eta': 0.01, 'learning_rate': 1e-06, 'weight_decay': 1e-06}
DPSH[ 1/150][08:27:37] bit:32, dataset:CIFAR, training... loss:12.15274
DPSH[ 2/150][08:27:52] bit:32, dataset:CIFAR, training....

In [ ]:
DSH12Cifar = {'eta': 0.05, 'learning_rate': 0.0001, 'weight_decay': 0.0001}
DSH24Cifar = {'eta': 0.1, 'learning_rate': 0.0001, 'weight_decay': 1e-05}
DSH32Cifar = {'eta': 0.1, 'learning_rate': 0.0001, 'weight_decay': 0.0001}
DSH48Cifar = {'eta': 0.01, 'learning_rate': 0.0001, 'weight_decay': 1e-06}

In [ ]:
DSH12Nus = {'eta': 0.05, 'learning_rate': 0.0001, 'weight_decay': 1e-05}
DSH24Nus = {'eta': 0.01, 'learning_rate': 0.0001, 'weight_decay': 1e-06}
DSH32Nus = {'eta': 0.01, 'learning_rate': 0.0001, 'weight_decay': 1e-06}
DSH48Nus = {'eta': 0.01, 'learning_rate': 0.0001, 'weight_decay': 1e-06}

In [ ]:
DSH12Img = {'eta': 0.01, 'learning_rate': 0.0001, 'weight_decay': 1e-06}
DSH24Img = {'eta': 0.01, 'learning_rate': 0.0001, 'weight_decay': 1e-06}
